In [2]:
import requests as rq
import time
import random
import lxml
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

/Users/boostigooff/.pyenv/versions/3.7.1/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


## Movie

In [3]:
response = rq.get('http://www.hogwartsishere.com/library/book/7391/')
doc = BeautifulSoup(response.text)


/Users/boostigooff/.pyenv/versions/3.7.1/lib/python3.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/boostigooff/.pyenv/versions/3.7.1/lib/python3.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
episodes = []
results = doc.find_all('li', class_="hover-white")
for result in results:
    episode = result.find('a')['href']
    episodes.append(episode)
episodes

['/library/book/7391/chapter/1/',
 '/library/book/7391/chapter/2/',
 '/library/book/7391/chapter/3/',
 '/library/book/7391/chapter/4/',
 '/library/book/7391/chapter/5/',
 '/library/book/7391/chapter/6/',
 '/library/book/7391/chapter/7/',
 '/library/book/7391/chapter/8/']

In [5]:
full_urls = []
for url in episodes:
    url = f"http://www.hogwartsishere.com/{url}"
    full_urls.append(url)
full_urls

['http://www.hogwartsishere.com//library/book/7391/chapter/1/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/2/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/3/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/4/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/5/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/6/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/7/',
 'http://www.hogwartsishere.com//library/book/7391/chapter/8/']

In [6]:
urltest = "http://www.hogwartsishere.com//library/book/7391/chapter/1/"
response = rq.get(urltest)
doc = BeautifulSoup(response.content)
line_text = []
scripts = doc.find('div', class_="font-size-16 roboto")

print(scripts)

#for script in scripts:
#    line_text.append(script.contents)
#    print(line_text)

<div class="font-size-16 roboto">
                            	Scene:<br/><br/>A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.<br/><br/> <br/><br/>Dumbledore: I should have known that you would be here...Professor McGonagall.<br/><br/> <br/><br/>The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.<br/><br/> <br/><br/>McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?<br/><br/> <br/><br/>Dumbledore: I'm afraid so, Professor. The good, and the bad.<br/><br/>McGonagall: An

In [7]:
parsed = re.split(r'(<br/>\w.*?<br/>)', str(scripts))
parsed

['<div class="font-size-16 roboto">\n                            \tScene:<br/>',
 '<br/>A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.<br/>',
 '<br/> <br/>',
 '<br/>Dumbledore: I should have known that you would be here...Professor McGonagall.<br/>',
 '<br/> <br/>',
 '<br/>The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.<br/>',
 '<br/> <br/>',
 '<br/>McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?<br/>',
 '<br/> <br/>',
 "<br/>Dumbledore: I'm afraid so, Professo

In [8]:
clean_lines = []
for line in parsed:
    a = re.sub('(<br/>)', '', str(line))
    b = re.sub('(\n)', '', a)
    c = re.sub('(<div\s.*?\\t)', '', b)
    d = re.sub('</div>', '', c)
    clean_lines.append(d)
print(clean_lines)

['Scene:', 'A neighbourhood on a street called Privet Drive. An owl, sitting on the street sign flies off to reveal a mysterious appearing old man walking through a forest near the street. He stops at the start of the street and takes out a mechanical device and zaps all the light out of the lampposts. He puts away the device and a cat meows. The man, ALBUS DUMBLEDORE, looks down at the cat, which is a tabby and is sitting on a brick ledge.', ' ', 'Dumbledore: I should have known that you would be here...Professor McGonagall.', ' ', 'The cat meows, sniffs out and the camera pans back to a wall. The cats shadow is seen progressing into a human. There are footsteps and MINERVA MCGONAGALL is revealed.', ' ', 'McGonagall: Good evening, Professor Dumbledore. Are the rumours true, Albus?', ' ', "Dumbledore: I'm afraid so, Professor. The good, and the bad.", '', 'McGonagall: And the boy?', '', 'Dumbledore: Hagrid is bringing him.', '', 'McGonagall: Do you think it wise to trust Hagrid with so

In [23]:
clean_lines_ok = []
for line in clean_lines:
    if str(line) != '':
        if str(line) != ' ':
            clean_lines_ok.append(line)
clean_lines_ok[1005]
                  

'Harry: Whoa! {The plant begins to move towards them.} Oh. Ahh! {The plant ties them up.}'

In [66]:
Oculus_Reparo = []
for line in clean_lines_ok:
    number_Oculus_Reparo = line.count("Oculus Reparo")
    Oculus_Reparo.append(number_Oculus_Reparo)
Oculus_Reparo = sum(Oculus_Reparo)

In [68]:
Locomotor_Mortis = []
for line in clean_lines_ok:
    number_Locomotor_Mortis = line.count("Locomotor Mortis")
    Locomotor_Mortis.append(number_Locomotor_Mortis)
Locomotor_Mortis = sum(Locomotor_Mortis)

In [69]:
Wingardium_Leviosa = []
for line in clean_lines_ok:
    number_Wingardium_Leviosa = line.count("Wingardium Leviosa")
    Wingardium_Leviosa.append(number_Wingardium_Leviosa)
Wingardium_Leviosa = sum(Wingardium_Leviosa)

In [70]:
Petrificus_Totalus = []
for line in clean_lines_ok:
    number_Petrificus_Totalus = line.count("Petrificus Totalus")
    Petrificus_Totalus.append(number_Petrificus_Totalus)
Petrificus_Totalus= sum(Petrificus_Totalus)

In [71]:
Alohomora = []
for line in clean_lines_ok:
    number_Alohomora = line.count("Alohomora")
    Alohomora.append(number_Alohomora)
Alohomora = sum(Alohomora)

In [72]:
Lumus_Solem = []
for line in clean_lines_ok:
    number_Lumus_Solem = line.count("Lumus Solem")
    Lumus_Solem.append(number_Lumus_Solem)
Lumus_Solem = sum(Lumus_Solem)

In [76]:
df_movie = pd.DataFrame(list(zip(Lumus_Solem, Alohomora, Petrificus_Totalus, Wingardium_Leviosa)), columns =['Lumus Solem', 'Alohomora', 'Petrificus Totalus', 'Wingardium Leviosa', 'Locomotor_Mortis', 'Oculus_Reparo']) 
df_movie

TypeError: zip argument #1 must support iteration

## Book

In [43]:
pip install PyPDF2


sqlalchemy-migrate 0.12.0 has requirement SQLAlchemy>=0.9.6, but you'll have sqlalchemy 0.7.10 which is incompatible.
requests 2.22.0 has requirement chardet<3.1.0,>=3.0.2, but you'll have chardet 2.3.0 which is incompatible.
jsonschema 3.0.1 has requirement six>=1.11.0, but you'll have six 1.10.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install nltk

sqlalchemy-migrate 0.12.0 has requirement SQLAlchemy>=0.9.6, but you'll have sqlalchemy 0.7.10 which is incompatible.
requests 2.22.0 has requirement chardet<3.1.0,>=3.0.2, but you'll have chardet 2.3.0 which is incompatible.
jsonschema 3.0.1 has requirement six>=1.11.0, but you'll have six 1.10.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [45]:
import PyPDF2 

In [46]:
#write a for-loop to open many files -- leave a comment if you'd #like to learn how
filename = 'harry_potter_and_the_sorcerers_stone.pdf' 
#open allows you to read the file
pdfFileObj = open(filename,'rb')
#The pdfReader variable is a readable object that will be parsed
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#discerning the number of pages will allow us to parse through all #the pages
num_pages = pdfReader.numPages
count = 0
text = ""
#The while loop will read each page
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
#This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
if text != "":
    text = text
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
else:
    text = textract.process(fileurl, method='tesseract', language='eng')
# Now we have a text variable which contains all the text derived #from our PDF file. Type print(text) to see what it contains. It #likely contains a lot of spaces, possibly junk such as '\n' etc.
# Now, we will clean our text variable, and return it as a list of keywords.

In [47]:
text

' Harry Potter and the Sorcerer™s Stone  By J.K. Rowling \n   CHAPTER ONE \n  The Boy Who Lived  \n      Mr. and Mrs. Dursley, of number four, Privet\n Drive, were proud to say that they were perfectly normal, thank you very much. They were the last\n people you™d expect to be involved in anything strange or mysterious, because they just didn™t hold with such nonsense.  \n      Mr. Dursley was the director of a firm \ncalled Grunnings, which made drills. He was a big, beefy \nman with hardly any neck, although he did have \na very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she \nspent so much of her time craning over garden\n fences, spying on the neighbors. The Dursleys had a small son called Dudley and in thei\nr opinion there was no finer boy anywhere.  \n      The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was \nthat somebody would discover it. They didn™t

In [48]:
clean_lines2 = []
a = re.sub('(\n)', '', str(text))
b = re.sub('™', "'", a)
c = re.sub('Š', ' - ', b)

d = re.sub('(ﬁ)', '"', c)
e = re.sub('(ﬂ)', '"', d)
f = re.sub('(\s\s+)', ' ', e)
clean_lines2.append(f)
print(clean_lines2)

[' Harry Potter and the Sorcerer\'s Stone By J.K. Rowling CHAPTER ONE The Boy Who Lived Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you\'d expect to be involved in anything strange or mysterious, because they just didn\'t hold with such nonsense. Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere. The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn\'t think they could bear it if anyone found out abou

In [49]:
from collections import Counter

In [61]:
verbs_Harry = re.findall(r"\bHarry (\w*)", str(clean_lines2))
verbs_Harry

['Potter',
 '',
 'Potter',
 'survive',
 'to',
 'Potter',
 'Potter',
 '',
 'underneath',
 'in',
 'and',
 'off',
 'gently',
 'Potter',
 'Potter',
 'Potter',
 'woke',
 'heard',
 'groaned',
 'got',
 'was',
 'didn',
 'had',
 'had',
 'liked',
 'was',
 'needed',
 'must',
 'was',
 'often',
 'put',
 'and',
 'was',
 'hated',
 'as',
 'knew',
 'like',
 'put',
 'a',
 'aside',
 'and',
 'coming',
 'wasn',
 'had',
 'supposed',
 'were',
 'what',
 'thought',
 'had',
 'was',
 'felt',
 'moved',
 'stared',
 'a',
 'murmured',
 'asked',
 'peered',
 'read',
 'made',
 'in',
 'fell',
 'sat',
 'could',
 'had',
 'at',
 'was',
 'lay',
 'had',
 'furiously',
 'tried',
 'had',
 'Potter',
 'his',
 'was',
 'Hunting',
 'spent',
 'at',
 'watch',
 'didn',
 'went',
 'looked',
 'seriously',
 'get',
 'dodged',
 'picked',
 'saw',
 'went',
 'was',
 'and',
 'furiously',
 'didn',
 'and',
 'and',
 'could',
 'in',
 'angrily',
 '',
 'one',
 'sighed',
 'was',
 'right',
 'had',
 'walked',
 'turned',
 'leapt',
 'realized',
 'didn',
 '

In [51]:
verbs_Hermione = re.findall(r"\bHermione (\w*)", str(clean_lines2))
verbs_Hermione

['Granger',
 'Granger',
 'Granger',
 'in',
 'Granger',
 'whisper',
 'almost',
 'were',
 'Granger',
 'a',
 'Granger',
 'stretched',
 'stood',
 'does',
 'Granger',
 'Granger',
 'Granger',
 'Granger',
 'ignored',
 'Granger',
 'snapped',
 'wasn',
 'had',
 'was',
 'caught',
 'and',
 'opened',
 'gasped',
 'said',
 'snarled',
 'had',
 'had',
 'showed',
 'was',
 'was',
 'marched',
 'Granger',
 'was',
 'snap',
 'rolled',
 'didn',
 'was',
 'out',
 'Granger',
 'had',
 'who',
 'had',
 'Granger',
 'hung',
 'was',
 'left',
 'Granger',
 'as',
 'had',
 'moved',
 'sat',
 'was',
 'he',
 'joined',
 'had',
 'squeezed',
 'seized',
 'gasped',
 'had',
 'had',
 'and',
 'looked',
 'took',
 'had',
 'joined',
 'playing',
 'ever',
 'at',
 'suggested',
 'asked',
 'urged',
 'had',
 'jumped',
 'told',
 'ignored',
 'were',
 'had',
 'muttered',
 'suddenly',
 'stood',
 'screamed',
 'in',
 'squeaked',
 'in',
 'would',
 'snapped',
 'next',
 'thoughtfully',
 'in',
 'went',
 'were',
 'didn',
 'had',
 'wouldn',
 'argued',
 

In [52]:
verbs_Ron = re.findall(r"\bRon (\w*)", str(clean_lines2))
verbs_Ron

['again',
 'blurted',
 'stared',
 'eagerly',
 'just',
 'found',
 'reached',
 'so',
 'up',
 'gasped',
 'stared',
 'had',
 'sounded',
 'was',
 'warned',
 'picked',
 'wouldn',
 'in',
 'looked',
 'Weasley',
 'muttered',
 'asked',
 'looked',
 'gave',
 'stood',
 'said',
 '',
 'leapt',
 'picking',
 'said',
 'darkly',
 'glared',
 'took',
 'were',
 'behind',
 'whispered',
 'groaned',
 'collapsed',
 'suddenly',
 'muttered',
 'if',
 'managed',
 'didn',
 'as',
 'exchanged',
 'kicked',
 'and',
 'pretended',
 'were',
 'told',
 'telling',
 'hadn',
 'walked',
 'reasonably',
 'would',
 'had',
 'couldn',
 'prodding',
 'jumped',
 'were',
 'and',
 'what',
 'had',
 'was',
 'and',
 'casually',
 'suggested',
 'muttered',
 'had',
 'muttered',
 'furiously',
 'through',
 'loudly',
 'looked',
 'exactly',
 'whispered',
 'around',
 'moaned',
 'finally',
 'stared',
 'were',
 'thought',
 'in',
 'to',
 'moaned',
 'couldn',
 'grinned',
 'could',
 'headed',
 'to',
 'sighed',
 'or',
 'snarled',
 'was',
 'overheard',
 'l

In [53]:
Counter(verbs_Harry).most_common(30)

[('and', 69),
 ('had', 58),
 ('was', 45),
 ('Potter', 29),
 ('could', 22),
 ('looked', 22),
 ('didn', 21),
 ('felt', 20),
 ('couldn', 17),
 ('thought', 16),
 ('asked', 16),
 ('', 12),
 ('told', 12),
 ('said', 11),
 ('tried', 10),
 ('saw', 10),
 ('heard', 9),
 ('as', 9),
 ('knew', 9),
 ('turned', 9),
 ('noticed', 9),
 ('in', 8),
 ('went', 8),
 ('whispered', 8),
 ('stared', 7),
 ('to', 6),
 ('got', 6),
 ('sat', 6),
 ('pulled', 6),
 ('shook', 6)]

In [54]:
Counter(verbs_Hermione).most_common(30)

[('Granger', 17),
 ('had', 17),
 ('was', 11),
 ('in', 6),
 ('were', 5),
 ('and', 5),
 ('didn', 5),
 ('looked', 4),
 ('stood', 3),
 ('gasped', 3),
 ('said', 3),
 ('screamed', 3),
 ('ignored', 2),
 ('snapped', 2),
 ('as', 2),
 ('joined', 2),
 ('seized', 2),
 ('took', 2),
 ('at', 2),
 ('suggested', 2),
 ('tried', 2),
 ('whisper', 1),
 ('almost', 1),
 ('a', 1),
 ('stretched', 1),
 ('does', 1),
 ('wasn', 1),
 ('caught', 1),
 ('opened', 1),
 ('snarled', 1)]

In [55]:
Counter(verbs_Ron).most_common(30)

[('and', 30),
 ('in', 10),
 ('was', 9),
 ('had', 8),
 ('muttered', 7),
 ('said', 7),
 ('were', 5),
 ('looked', 4),
 ('whispered', 4),
 ('didn', 4),
 ('to', 4),
 ('stared', 3),
 ('asked', 3),
 ('couldn', 3),
 ('moaned', 3),
 ('thought', 3),
 ('sighed', 3),
 ('went', 3),
 ('gasped', 2),
 ('warned', 2),
 ('wouldn', 2),
 ('gave', 2),
 ('stood', 2),
 ('', 2),
 ('suddenly', 2),
 ('as', 2),
 ('would', 2),
 ('finally', 2),
 ('could', 2),
 ('pulled', 2)]

In [56]:
Alohomora = re.findall(r"(Alohomora)", str(clean_lines2))
Counter(Alohomora)


Counter({'Alohomora': 2})

In [57]:
Locomotor_Mortis = re.findall(r"(Locomotor Mortis)", str(clean_lines2))
Counter(Locomotor_Mortis)

Counter({'Locomotor Mortis': 1})

In [58]:
Petrificus_Totalus = re.findall(r"(Petrificus Totalus)", str(clean_lines2))
Counter(Petrificus_Totalus)

Counter({'Petrificus Totalus': 1})

In [59]:
Wingardium_Leviosa = re.findall(r"(Wingardium Leviosa)", str(clean_lines2))
Counter(Wingardium_Leviosa)

Counter({'Wingardium Leviosa': 3})

In [62]:
Occulus_Reparo = re.findall(r"(Occulus Reparo)", str(clean_lines2))
Counter(Occulus_Reparo)

Counter()